In [1]:
import time
import unittest
from test.test_meteorology import Test_relhum

import cupy as cp
import dask
import numpy as np
import pandas as pd
import src.geocat.comp.meteorology as geo
import xarray as xr
xp = np
p_def = xp.random.uniform(low=2000,high=100800,size=10**6)
t_def = xp.random.uniform(low=194.65,high=302.45,size=10**6)
q_def = xp.random.uniform(low=0,high=0.02038,size=10**6)

In [2]:
import dask.distributed as dd
client = dd.Client()

2022-06-28 10:49:31,678 - distributed.diskutils - INFO - Found stale lock file and directory '/glade/u/home/hkashgar/geocat-comp/dask-worker-space/worker-c1brgcfz', purging


In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/hkashgar/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/hkashgar/proxy/8787/status,Workers: 1
Total threads: 1,Total memory: 20.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42672,Workers: 1
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/hkashgar/proxy/8787/status,Total threads: 1
Started: Just now,Total memory: 20.00 GiB
Comm: tcp://127.0.0.1:42854,Total threads: 1
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/hkashgar/proxy/33283/status,Memory: 20.00 GiB
Nanny: tcp://127.0.0.1:44004,


## Testing Numpy input and CPU

In [4]:
rh_gt_2 = geo.relhum(t_def, q_def, p_def, use_gpu = False)

step 1: conversion:
5.9604644775390625e-06
step 2: clip
0.0021424293518066406
step 3: it
0.005689382553100586
step 4: t2
0.0012912750244140625
step 5: es
0.019776344299316406
step 6: rh
0.005697727203369141
step 7: rh
0.0008924007415771484


## Testing Numpy input and GPU

In [5]:
rh_gt_2 = geo.relhum(t_def, q_def, p_def, use_gpu = True)

step 1: conversion:
0.020243406295776367
step 2: clip
0.10709118843078613
step 3: it
0.003403902053833008
step 4: t2
0.0008230209350585938
step 5: es
0.005720853805541992
step 6: rh
0.001428365707397461
step 7: rh
7.414817810058594e-05


## Testing Xarray input and CPU

In [6]:
p = xr.DataArray(p_def)
t = xr.DataArray(t_def)
q = xr.DataArray(q_def)
a = geo.relhum(t, q, p, use_gpu = False)

step 1: conversion:
1.6689300537109375e-06
step 2: clip
0.0036063194274902344
step 3: it
0.00741124153137207
step 4: t2
0.001668691635131836
step 5: es
0.23754167556762695
step 6: rh
0.03779482841491699
step 7: rh
0.0008211135864257812


## Testing Xarray input and GPU

In [7]:
p = xr.DataArray(p_def)
t = xr.DataArray(t_def)
q = xr.DataArray(q_def)
a = geo.relhum(t, q, p, use_gpu = True)

step 1: conversion:
0.005831241607666016
step 2: clip
0.0006158351898193359
step 3: it
0.0001316070556640625
step 4: t2
5.4836273193359375e-05
step 5: es
0.17171478271484375
step 6: rh
0.00488591194152832
step 7: rh
0.0008091926574707031


## Testing Dask input with CPU

In [8]:
p = xr.DataArray(p_def).chunk(10)
t = xr.DataArray(t_def).chunk(10)
q = xr.DataArray(q_def).chunk(10)
a = geo.relhum(t, q, p, use_gpu = False)

step 1: conversion:
8.058547973632812e-05
step 2: clip
0.017906665802001953
step 3: it
0.06217646598815918
step 4: t2
0.018827199935913086
step 5: es
0.30895113945007324
step 6: rh
0.11308002471923828
step 7: rh
0.019645214080810547


## Testing Dask input with GPU

In [9]:
p = xr.DataArray(p_def).chunk(10)
t = xr.DataArray(t_def).chunk(10)
q = xr.DataArray(q_def).chunk(10)
a = geo.relhum(t, q, p, use_gpu = True)

step 1: conversion:
0.050879478454589844
step 2: clip
0.019103050231933594
step 3: it
0.03528285026550293
step 4: t2
0.020735740661621094
step 5: es
0.24248147010803223
step 6: rh
0.12958788871765137
step 7: rh
0.04177594184875488


In [10]:
maxtemp = 375.16
mintemp = 173.16
t = xp.random.uniform(low=194.65,high=302.45,size=10**3)
t = xr.DataArray(cp.asarray(t)).chunk(10)
# replace values of t above and below max and min values for temperature
type(t)

xarray.core.dataarray.DataArray

## Unittests with CPU!

In [ ]:
test = Test_relhum()

test.setUpClass()
test.test_float_input(use_gpu = False)
test.test_list_input(use_gpu = False)
test.test_numpy_input(use_gpu = False)
test.test_dims_error(use_gpu = False)
test.test_xarray_type_error(use_gpu = False)
test.test_dask_compute(use_gpu = False)
test.test_dask_lazy(use_gpu = False)

## Unittests with GPU

In [ ]:
test = Test_relhum()

test.setUpClass()
test.test_float_input(use_gpu = True)
test.test_list_input(use_gpu = True)
test.test_numpy_input(use_gpu = True)
test.test_dims_error(use_gpu = True)
test.test_xarray_type_error(use_gpu = True)
test.test_dask_compute(use_gpu = True)
test.test_dask_lazy(use_gpu = True)

## Benchmark Results for different Array Sizes

In [12]:
def test_numpy_input(use_gpu):
    p = np.asarray(p_def)
    t = np.asarray(t_def)
    q = np.asarray(q_def)
    
    assert xp.allclose(geo.relhum(t, q, p,use_gpu), rh_gt_2, atol=0.1)

In [ ]:
#Test_relhum main body
Routine = "Relhum"
print(Routine)
allData = pd.DataFrame()
#For different Array sizes
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    #for numpy and cupy both
    for xp in [np,cp]:
        #calculation will be repeated 10 time to get the less biased performance results
        repeat = np.zeros([10])
        repsize = 10
        for rep in range(0,repsize):
            #create different sizes of arrays
            p_def = np.random.uniform(low=2000,high=100800,size=10**i)
            t_def = np.random.uniform(low=194.65,high=302.45,size=10**i)
            q_def = np.random.uniform(low=0,high=0.02038,size=10**i)
            if(xp == cp):
                geo.relhum(t_def, q_def, p_def,True)
                cp.cuda.runtime.deviceSynchronize()
                time1 = time.time()
                geo.relhum(t_def, q_def, p_def,True)
                cp.cuda.runtime.deviceSynchronize()
                time2 = time.time()
                repeat[rep] = time2-time1
            else:
                time1 = time.time()
                geo.relhum(t_def, q_def,p_def,False)
                time2 = time.time()
                repeat[rep] = time2-time1
        #save times
        data = {'Routine': np.repeat(Routine, repsize),
                'Approach': np.repeat(xp.__name__ , repsize),
                'ArraySize': np.repeat(ArraySize , repsize),
                'iteration' : np.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,np.mean(repeat), "seconds")

## Plotting

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

arraysizes = np.unique(allData['ArraySize'])
sd_numpy = np.zeros(len(arraysizes))
sd_cupy = np.zeros(len(arraysizes))
y_numpy = np.zeros(len(arraysizes))
y_cupy = np.zeros(len(arraysizes))

for i in range(0,len(arraysizes)):
    cupydata = allData.loc[(allData['ArraySize'] == arraysizes[i]) & (allData['Approach'] == 'cupy')]
    numpydata = allData.loc[(allData['ArraySize'] == arraysizes[i]) & (allData['Approach'] == 'numpy')]
    y_cupy[i] = np.mean(cupydata['Runtime(s)'])
    y_numpy[i] = np.mean(numpydata['Runtime(s)'])
    sd_cupy[i] = np.std(cupydata['Runtime(s)'])
    sd_numpy[i] = np.std(numpydata['Runtime(s)'])

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
ax.errorbar(arraysizes, y_numpy, yerr=sd_numpy, fmt='-o')
ax.errorbar(arraysizes, y_cupy, yerr=sd_cupy, fmt='-o')
ax.plot(arraysizes, y_numpy, label='numpy',color="steelblue")
ax.plot(arraysizes, y_cupy, label='cupy',color="darkorange")  # Plot more data on the axes...
ax.legend();  # Add a legend.
ax.set_xlabel('ArraySize')  # Add an x-label to the axes.
ax.set_ylabel('Runtime(s)')  # Add a y-label to the axes.
ax.set_title(("Test_relhum"))
ax.set_xscale('log')
ax.set_yscale('log')
#plt.savefig("Test_relhum.jpg",dpi=fig.dpi)

In [ ]:
import dask.distributed as dd

client = dd.Client()

In [ ]:
client

## Benchmark Results for different ArraySizes Xarray

In [ ]:
#Test_relhum main body
Routine = "Relhum"
print(Routine)
allData = pd.DataFrame()
#For different Array sizes
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    #for numpy and cupy both
    for xp in [np,cp]:
        #calculation will be repeated 10 time to get the less biased performance results
        repeat = np.zeros([10])
        repsize = 10
        for rep in range(0,repsize):
            #create different sizes of arrays
            p_def = xr.DataArray(np.random.uniform(low=2000,high=100800,size=10**i))
            t_def = xr.DataArray(np.random.uniform(low=194.65,high=302.45,size=10**i))
            q_def = xr.DataArray(np.random.uniform(low=0,high=0.02038,size=10**i))
            if(xp == cp):
                geo.relhum(t_def, q_def, p_def,True)
                cp.cuda.runtime.deviceSynchronize()
                time1 = time.time()
                geo.relhum(t_def, q_def, p_def,True)
                cp.cuda.runtime.deviceSynchronize()
                time2 = time.time()
                repeat[rep] = time2-time1
            else:
                time1 = time.time()
                geo.relhum(t_def, q_def,p_def,False)
                time2 = time.time()
                repeat[rep] = time2-time1
        #save times
        data = {'Routine': np.repeat(Routine, repsize),
                'Approach': np.repeat(xp.__name__ , repsize),
                'ArraySize': np.repeat(ArraySize , repsize),
                'iteration' : np.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,np.mean(repeat), "seconds")

In [ ]:
#Test_relhum main body
Routine = "Relhum"
print(Routine)
allData = pd.DataFrame()
#For different Array sizes
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    #for numpy and cupy both
    for xp in [np,cp]:
        #calculation will be repeated 10 time to get the less biased performance results
        repeat = np.zeros([10])
        repsize = 10
        for rep in range(0,repsize):
            #create different sizes of arrays
            p_def = xr.DataArray(cp.random.uniform(low=2000,high=100800,size=10**i))
            t_def = xr.DataArray(cp.random.uniform(low=194.65,high=302.45,size=10**i))
            q_def = xr.DataArray(cp.random.uniform(low=0,high=0.02038,size=10**i))
            if(xp == cp):
                geo.relhum(t_def, q_def, p_def,True)
                cp.cuda.runtime.deviceSynchronize()
                time1 = time.time()
                geo.relhum(t_def, q_def, p_def,True)
                cp.cuda.runtime.deviceSynchronize()
                time2 = time.time()
                repeat[rep] = time2-time1
            else:
                time1 = time.time()
                geo.relhum(t_def, q_def,p_def,False)
                time2 = time.time()
                repeat[rep] = time2-time1
        #save times
        data = {'Routine': np.repeat(Routine, repsize),
                'Approach': np.repeat(xp.__name__ , repsize),
                'ArraySize': np.repeat(ArraySize , repsize),
                'iteration' : np.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,np.mean(repeat), "seconds")

In [ ]:
#Test_relhum main body
Routine = "Relhum"
print(Routine)
allData = pd.DataFrame()
#For different Array sizes
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    #for numpy and cupy both
    for xp in [np,cp]:
        #calculation will be repeated 10 time to get the less biased performance results
        repeat = np.zeros([10])
        repsize = 10
        for rep in range(0,repsize):
            #create different sizes of arrays
            p_def = xr.DataArray(np.random.uniform(low=2000,high=100800,size=10**i)).chunk(10)
            t_def = xr.DataArray(np.random.uniform(low=194.65,high=302.45,size=10**i)).chunk(10)
            q_def = xr.DataArray(np.random.uniform(low=0,high=0.02038,size=10**i)).chunk(10)
            if(xp == cp):
                geo.relhum(t_def, q_def, p_def,True)
                cp.cuda.runtime.deviceSynchronize()
                time1 = time.time()
                geo.relhum(t_def, q_def, p_def,True)
                cp.cuda.runtime.deviceSynchronize()
                time2 = time.time()
                repeat[rep] = time2-time1
            else:
                time1 = time.time()
                geo.relhum(t_def, q_def,p_def,False)
                time2 = time.time()
                repeat[rep] = time2-time1
        #save times
        data = {'Routine': np.repeat(Routine, repsize),
                'Approach': np.repeat(xp.__name__ , repsize),
                'ArraySize': np.repeat(ArraySize , repsize),
                'iteration' : np.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,np.mean(repeat), "seconds")